In [120]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from IPython.display import display

In [121]:
# Read data from .csv files 

loc = "voetbal_1956-20220309/"

raw_cards_file = pd.read_csv (loc + '_cards.csv')
raw_goals_file = pd.read_csv (loc + '_goals.csv')
raw_matches_file = pd.read_csv (loc + '_matches.csv')
raw_players_file = pd.read_csv (loc + '_players.csv')
raw_subs_file = pd.read_csv (loc + '_subs.csv')

In [122]:
# function to remove duplicates 
def Duplicates(rawName , colName: str, previousName: str, newName: str ):
        if colName == "club":
                rawName["home"].replace({previousName: newName }, inplace=True)
                rawName["away"].replace({previousName: newName }, inplace=True)
                rawName["team"].replace({previousName: newName }, inplace=True)
        elif colName == "match":
                rawName["home"].replace({previousName: newName }, inplace=True)
                rawName["away"].replace({previousName: newName }, inplace=True)
        elif colName == "card": 
                rawName["type"].replace({previousName: newName }, inplace=True)
        else:
                print("nothing")

In [123]:
# function used to get the last season
def toOneSeason(season):
    return season[5:len(season)]

# Goals

In [124]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

# to check the duplicates teams
ids = label.fit_transform(raw_goals_file['home'])
mappingHome = dict(zip(label.classes_, range(len(label.classes_))))

In [125]:
Duplicates(raw_goals_file, "club", "afc-ajax", "ajax-amsterdam")
Duplicates(raw_goals_file, "club", "feyenoord", "feyenoord-rotterdam")
Duplicates(raw_goals_file, "club", "vitesse", "vitesse-arnheim")
Duplicates(raw_goals_file, "club", "willem-ii", "willem-ii-tilburg")
Duplicates(raw_goals_file, "club", "mvv", "mvv-maastricht")
Duplicates(raw_goals_file, "club", "sbv-excelsior", "excelsior-rotterdam")
Duplicates(raw_goals_file, "club", "sc-cambuur", "cambuur-leeuwarden")
Duplicates(raw_goals_file, "club", "fc-twente", "twente-enschede")

In [126]:
def firstGoal(col: str):
    return raw_goals_file[col].loc[(raw_goals_file['new_score']== '1:0') | (raw_goals_file['new_score'] == '0:1')]

In [127]:
df_goals = pd.DataFrame()
season = firstGoal("saison")
round =  firstGoal("round")
home =  firstGoal("home")
away = firstGoal("away")
# team = firstGoal("team")
first_goal = firstGoal("new_score")

df_goals['season'] = season
df_goals['round'] = round
df_goals['home'] = home
df_goals['away'] = away
# df_goals['team'] = team
df_goals['first_goal'] = first_goal

df_goals.loc[(df_goals["first_goal"] == "1:0"), "first_goal"] = "home"
df_goals.loc[(df_goals["first_goal"] == "0:1"), "first_goal"] = "away"

df_goals['season'] = df_goals['season'].apply(toOneSeason)
df_goals['season'] = df_goals['season'].astype(int)

df_goals

,season,round,home,away,first_goal
0,1957,16,ajax-amsterdam,bvc-amsterdam,home
5,1957,27,ajax-amsterdam,fc-den-bosch,home
6,1957,15,ajax-amsterdam,fc-utrecht,home
9,1957,25,ajax-amsterdam,feyenoord-rotterdam,home
10,1957,13,ajax-amsterdam,fortuna-54,away
...,...,...,...,...,...
60854,2022,3,willem-ii-tilburg,pec-zwolle,home
60855,2022,7,willem-ii-tilburg,psv-eindhoven,home
60858,2022,21,willem-ii-tilburg,rkc-waalwijk,home
60862,2022,16,willem-ii-tilburg,cambuur-leeuwarden,away


# Matches

In [128]:
#Function used for the matches:
# 1 means that home team is the winner 
# -1 means that home team is the loser
# 0 means that home team tied with the away team
def ClassifyMatches(row):
    if(row.home_score > row.away_score):
        return 1
    elif(row.home_score < row.away_score):
        return -1
    else:
        return 0

In [129]:
# to check the duplicates
ids = label.fit_transform(raw_matches_file['home'])
mappingHome = dict(zip(label.classes_, range(len(label.classes_))))

In [130]:
Duplicates(raw_matches_file, "match", "afc-ajax", "ajax-amsterdam")
Duplicates(raw_matches_file, "match", "feyenoord", "feyenoord-rotterdam")
Duplicates(raw_matches_file, "match", "vitesse", "vitesse-arnheim")
Duplicates(raw_matches_file, "match", "willem-ii", "willem-ii-tilburg")
Duplicates(raw_matches_file, "match", "mvv", "mvv-maastricht")
Duplicates(raw_matches_file, "match", "sbv-excelsior", "excelsior-rotterdam")
Duplicates(raw_matches_file, "match", "sc-cambuur", "cambuur-leeuwarden")
Duplicates(raw_matches_file, "match", "fc-twente", "twente-enschede")

In [131]:
df_matches = pd.DataFrame()
season = raw_matches_file["saison"]
round =  raw_matches_file["round"]
home =  raw_matches_file["home"]
away = raw_matches_file["away"]
home_score = raw_matches_file["home_score"]
away_score = raw_matches_file["away_score"]

df_matches['season'] = season
df_matches['round'] = round
df_matches['home'] = home
df_matches['away'] = away
df_matches['home_score'] = home_score
df_matches['away_score'] = away_score
df_matches['winning_team'] = df_matches.apply(lambda row: ClassifyMatches(row),axis=1)


df_matches['season'] = df_matches['season'].apply(toOneSeason)
df_matches['season'] = df_matches['season'].astype(int)

df_matches

,season,round,home,away,home_score,away_score,winning_team
0,1957,16,ajax-amsterdam,bvc-amsterdam,2,3,-1
1,1957,27,ajax-amsterdam,fc-den-bosch,1,0,1
2,1957,15,ajax-amsterdam,fc-utrecht,1,2,-1
3,1957,25,ajax-amsterdam,feyenoord-rotterdam,1,0,1
4,1957,13,ajax-amsterdam,fortuna-54,0,2,-1
...,...,...,...,...,...,...,...
19925,2022,21,willem-ii-tilburg,rkc-waalwijk,3,1,1
19926,2022,16,willem-ii-tilburg,cambuur-leeuwarden,1,3,-1
19927,2022,25,willem-ii-tilburg,sc-heerenveen,0,0,0
19928,2022,12,willem-ii-tilburg,sparta-rotterdam,0,3,-1


# Merge Goals and Matches 

In [132]:
df_merge= pd.merge(df_goals,df_matches,how='inner',left_on=['season','round','home', 'away'],right_on=['season','round','home', 'away'])
df_merge

,season,round,home,away,first_goal,home_score,away_score,winning_team
0,1957,16,ajax-amsterdam,bvc-amsterdam,home,2,3,-1
1,1957,27,ajax-amsterdam,fc-den-bosch,home,1,0,1
2,1957,15,ajax-amsterdam,fc-utrecht,home,1,2,-1
3,1957,25,ajax-amsterdam,feyenoord-rotterdam,home,1,0,1
4,1957,13,ajax-amsterdam,fortuna-54,away,0,2,-1
...,...,...,...,...,...,...,...,...
18394,2022,3,willem-ii-tilburg,pec-zwolle,home,1,0,1
18395,2022,7,willem-ii-tilburg,psv-eindhoven,home,2,1,1
18396,2022,21,willem-ii-tilburg,rkc-waalwijk,home,3,1,1
18397,2022,16,willem-ii-tilburg,cambuur-leeuwarden,away,1,3,-1


# Prediction 

In [133]:
df_prediction = pd.DataFrame()
season = df_merge["season"]
round =  df_merge["round"]
home =  df_merge["home"]
away = df_merge["away"]
first_goal = df_merge["first_goal"]
home_score = df_merge["home_score"]
away_score = df_merge["away_score"]
winning_team = df_merge["winning_team"]

df_prediction['season'] = season
df_prediction['round'] = round
df_prediction['home'] = home
df_prediction['away'] = away
df_prediction["first_goal"] = first_goal
df_prediction['home_score'] = home_score
df_prediction['away_score'] = away_score
df_prediction["winning_team"] = winning_team

In [134]:
from sklearn import tree
from sklearn.model_selection import train_test_split

label = LabelEncoder()

In [146]:
ids = label.fit_transform(df_merge['home'])
mappingHome = dict(zip(label.classes_, range(len(label.classes_))))
print(list(mappingHome.keys())[list(mappingHome.values()).index(2)])

df_prediction["home"] = label.fit_transform(df_prediction['home'])
df_prediction["away"] = label.fit_transform(df_prediction['away'])
df_prediction["first_goal"] = label.fit_transform(df_prediction['first_goal'])

print(mappingHome)

ajax-amsterdam
{'ado-den-haag': 0, 'afc-dws': 1, 'ajax-amsterdam': 2, 'az-alkmaar': 3, 'blauw-wit': 4, 'bvc-amsterdam': 5, 'cambuur-leeuwarden': 6, 'de-graafschap': 7, 'de-volewijckers': 8, 'excelsior-rotterdam': 9, 'fc-amsterdam': 10, 'fc-den-bosch': 11, 'fc-dordrecht': 12, 'fc-eindhoven': 13, 'fc-emmen': 14, 'fc-groningen': 15, 'fc-utrecht': 16, 'fc-volendam': 17, 'fc-wageningen': 18, 'feyenoord-rotterdam': 19, 'fortuna-54': 20, 'fortuna-sittard': 21, 'go-ahead-eagles': 22, 'helmond-sport': 23, 'heracles-almelo': 24, 'hfc-haarlem': 25, 'holland-sport': 26, 'mvv-maastricht': 27, 'nac-breda': 28, 'nec-nijmegen': 29, 'noad-tilburg': 30, 'pec-zwolle': 31, 'psv-eindhoven': 32, 'rbc-roosendaal': 33, 'rkc-waalwijk': 34, 'roda-jc-kerkrade': 35, 'sc-heerenveen': 36, 'sc-telstar': 37, 'sc-veendam': 38, 'sittardia': 39, 'sparta-rotterdam': 40, 'svv-schiedam': 41, 'twente-enschede': 42, 'usv-elinkwijk': 43, 'vitesse-arnheim': 44, 'vvv-venlo': 45, 'willem-ii-tilburg': 46, 'xerxes-dzb': 47}


In [138]:
X = df_prediction[["home","away","first_goal"]]
y = df_prediction["winning_team"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [139]:
winning_team = tree.DecisionTreeClassifier()
winning_team.fit(X_train,y_train)

winning_team.score(X_test,y_test)

0.6342391304347826

In [151]:
winning_team.predict([[32, 2 , 1]])

array([1], dtype=int64)